# Import Modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Prepare Data

In [ ]:
def prepare_data(data_csv):
  """Read data"""
  df = pd.read_csv(data_csv)
  df['Date/Time'] = pd.to_datetime(df['Date/Time'])
  df = df.sort_values(by='Date/Time')
  return df

def create_seq(data, seq_len):
  """Create data sequence for LSTM with sequence length"""
  sequences = []
  targets = []

  for i in range(len(data)-seq_len):
    seq = data[i:i+seq_len]
    target = data[i+seq_len]
    sequences.append(seq)
    targets.append(target)

  return np.array(sequences), np.array(targets)

# Build models

In [ ]:
def LSTM_model(seq_len, num_features):
  """
  Build LSTM model:
  - 2 layers of LSTM with 50 units
  - Dropout 0.2 to avoid overfitting
  - Dense layer to predict price fluctuations
  """
  model = Sequential([
      LSTM(50, activation='relu', input_shape=(seq_len, num_features), return_sequences=True),
      Dropout(0.2),
      LSTM(50, activation='relu', return_sequences=False),
      Dropout(0.2),
      Dense(1, activation='linear')
  ])
  model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
  return model

# Processing Data And Training

In [ ]:
def train(data_csv, seq_len=15, epochs=20):
  """Processing data:
    - Use 5 features: Open, High, Low, Close and Volume
    - Normalize data using MinMaxScaler
    - Create a data series with a length of 15 sessions for prediction
    Train Model:
    - Divide 80% of the data for training
    - Divide 20% of the data for testing
    - 20 epochs with batch size 32
    - Use Mean Squared Error (MSE) as the loss function
  """
  df = prepare_data(data_csv)

  features = ['Open', 'High', 'Low', 'Close', 'Volume']
  data = df[features].values

  scaler = MinMaxScaler()
  scaled_data = scaler.fit_transform(data)

  X, y = create_seq(scaled_data, seq_len)

  train_size = int(len(X) * 0.8)
  X_train, y_train = X[:train_size], y[:train_size]
  X_test, y_test = X[train_size:], y[train_size:]

  model = LSTM_model(seq_len, len(features))
  history = model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_data=(X_test, y_test))

  train_pred = model.predict(X_train)
  test_pred = model.predict(X_test)

  return model, scaler, train_pred, test_pred, history, y_train, y_test, X_train, X_test

# Performance

## FPT

In [ ]:
data_fpt = "/content/FPT.csv"
model, scaler, train_pred, test_pred, history, y_train, y_test, X_train, X_test= train(data_fpt)

# Results
# MSE (Mean Squared Error) is an important index to evaluate the accuracy of a prediction model
print("\Resultsá:")
print(f"MSE on the train set: {np.mean((train_pred - y_train) ** 2):.6f}")
print(f"MSE on the test set: {np.mean((test_pred - y_test) ** 2):.6f}")

# Predict price movements for the next session
last_sequence = X_test[-1:]
next_price_change = model.predict(last_sequence)[0][0]
print(f"\nPredict price movements for the next session: {next_price_change:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
2435/2435 ━━━━━━━━━━━━━━━━━━━━ 46s 17ms/step - loss: 0.0446 - val_loss: 0.0979
Epoch 2/20
2435/2435 ━━━━━━━━━━━━━━━━━━━━ 44s 18ms/step - loss: 0.0402 - val_loss: 0.0987
Epoch 3/20
2435/2435 ━━━━━━━━━━━━━━━━━━━━ 42s 17ms/step - loss: 0.0403 - val_loss: 0.0978
Epoch 4/20
2435/2435 ━━━━━━━━━━━━━━━━━━━━ 44s 18ms/step - loss: 0.0402 - val_loss: 0.0980
Epoch 5/20
2435/2435 ━━━━━━━━━━━━━━━━━━━━ 82s 18ms/step - loss: 0.0401 - val_loss: 0.0983
Epoch 6/20
2435/2435 ━━━━━━━━━━━━━━━━━━━━ 43s 18ms/step - loss: 0.0402 - val_loss: 0.0982
Epoch 7/20
2435/2435 ━━━━━━━━━━━━━━━━━━━━ 80s 17ms/step - loss: 0.0401 - val_loss: 0.0978
Epoch 8/20
2435/2435 ━━━━━━━━━━━━━━━━━━━━ 84s 18ms/step - loss: 0.0401 - val_loss: 0.0980
Epoch 9/20
2435/2435 ━━━━━━━━━━━━━━━━━━━━ 82s 18ms/step - loss: 0.0402 - val_loss: 0.0979
Epoch 10/20
2435/2435 ━━━━━━━━━━━━━━━━━━━━ 85s 19ms/step - loss: 0.0401 - val_loss: 0.0978
Epoch 11/20
2435/2435 ━━━━━━━━━━━━━━━━━━━━ 79s 18ms/step - loss: 0.0400 - val_loss: 0.0981
Epoch 12

## MSN

In [ ]:
data_msn = "/content/MSN.csv"
model, scaler, train_pred, test_pred, history, y_train, y_test, X_train, X_test= train(data_msn)

# Results
print("\Resultsá:")
print(f"MSE on the train set: {np.mean((train_pred - y_train) ** 2):.6f}")
print(f"MSE on the test set: {np.mean((test_pred - y_test) ** 2):.6f}")

# Predict price movements for the next session
last_sequence = X_test[-1:]
next_price_change = model.predict(last_sequence)[0][0]
print(f"\nPredict price movements for the next session: {next_price_change:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
3384/3384 ━━━━━━━━━━━━━━━━━━━━ 63s 17ms/step - loss: 0.0399 - val_loss: 0.0184
Epoch 2/20
3384/3384 ━━━━━━━━━━━━━━━━━━━━ 58s 17ms/step - loss: 0.0366 - val_loss: 0.0184
Epoch 3/20
3384/3384 ━━━━━━━━━━━━━━━━━━━━ 81s 17ms/step - loss: 0.0366 - val_loss: 0.0184
Epoch 4/20
3384/3384 ━━━━━━━━━━━━━━━━━━━━ 84s 18ms/step - loss: 0.0367 - val_loss: 0.0184
Epoch 5/20
3384/3384 ━━━━━━━━━━━━━━━━━━━━ 82s 18ms/step - loss: 0.0366 - val_loss: 0.0184
Epoch 6/20
3384/3384 ━━━━━━━━━━━━━━━━━━━━ 80s 17ms/step - loss: 0.0366 - val_loss: 0.0184
Epoch 7/20
3384/3384 ━━━━━━━━━━━━━━━━━━━━ 84s 18ms/step - loss: 0.0366 - val_loss: 0.0185
Epoch 8/20
3384/3384 ━━━━━━━━━━━━━━━━━━━━ 82s 18ms/step - loss: 0.0365 - val_loss: 0.0184
Epoch 9/20
3384/3384 ━━━━━━━━━━━━━━━━━━━━ 58s 17ms/step - loss: 0.0366 - val_loss: 0.0185
Epoch 10/20
3384/3384 ━━━━━━━━━━━━━━━━━━━━ 82s 17ms/step - loss: 0.0368 - val_loss: 0.0184
Epoch 11/20
3384/3384 ━━━━━━━━━━━━━━━━━━━━ 58s 17ms/step - loss: 0.0367 - val_loss: 0.0184
Epoch 12

## PNJ

In [ ]:
data_pnj = "/content/PNJ.csv"
model, scaler, train_pred, test_pred, history, y_train, y_test, X_train, X_test= train(data_pnj)

# Results
print("\Resultsá:")
print(f"MSE on the train set: {np.mean((train_pred - y_train) ** 2):.6f}")
print(f"MSE on the test set: {np.mean((test_pred - y_test) ** 2):.6f}")

# Predict price movements for the next session
last_sequence = X_test[-1:]
next_price_change = model.predict(last_sequence)[0][0]
print(f"\nPredict price movements for the next session: {next_price_change:.4f}")

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3133/3133 ━━━━━━━━━━━━━━━━━━━━ 58s 17ms/step - loss: 0.0531 - val_loss: 0.0231
Epoch 2/20
3133/3133 ━━━━━━━━━━━━━━━━━━━━ 53s 17ms/step - loss: 0.0499 - val_loss: 0.0230
Epoch 3/20
3133/3133 ━━━━━━━━━━━━━━━━━━━━ 53s 17ms/step - loss: 0.0499 - val_loss: 0.0231
Epoch 4/20
3133/3133 ━━━━━━━━━━━━━━━━━━━━ 84s 18ms/step - loss: 0.0499 - val_loss: 0.0230
Epoch 5/20
3133/3133 ━━━━━━━━━━━━━━━━━━━━ 81s 17ms/step - loss: 0.0498 - val_loss: 0.0230
Epoch 6/20
3133/3133 ━━━━━━━━━━━━━━━━━━━━ 81s 17ms/step - loss: 0.0498 - val_loss: 0.0230
Epoch 7/20
3133/3133 ━━━━━━━━━━━━━━━━━━━━ 83s 17ms/step - loss: 0.0498 - val_loss: 0.0230
Epoch 8/20
3133/3133 ━━━━━━━━━━━━━━━━━━━━ 82s 17ms/step - loss: 0.0498 - val_loss: 0.0230
Epoch 9/20
3133/3133 ━━━━━━━━━━━━━━━━━━━━ 82s 17ms/step - loss: 0.0499 - val_loss: 0.0230
Epoch 10/20
3133/3133 ━━━━━━━━━━━━━━━━━━━━ 82s 18ms/step - loss: 0.0498 - val_loss: 0.0230
Epoch 11/20
3133/3133 ━━━━━━━━━━━━━━━━━━━━ 80s 17ms/step - loss: 0.0498 - val_loss: 0.0230
Epoch 12/20
3133/31

## VIC

In [ ]:
data_vic = "/content/VIC.csv"
model, scaler, train_pred, test_pred, history, y_train, y_test, X_train, X_test= train(data_vic)

# Results
print("\Resultsá:")
print(f"MSE on the train set: {np.mean((train_pred - y_train) ** 2):.6f}")
print(f"MSE on the test set: {np.mean((test_pred - y_test) ** 2):.6f}")

# Predict price movements for the next session
last_sequence = X_test[-1:]
next_price_change = model.predict(last_sequence)[0][0]
print(f"\nPredict price movements for the next session: {next_price_change:.4f}")

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2532/2532 ━━━━━━━━━━━━━━━━━━━━ 50s 18ms/step - loss: 0.0895 - val_loss: 0.0447
Epoch 2/20
2532/2532 ━━━━━━━━━━━━━━━━━━━━ 82s 18ms/step - loss: 0.0792 - val_loss: 0.0437
Epoch 3/20
2532/2532 ━━━━━━━━━━━━━━━━━━━━ 44s 17ms/step - loss: 0.0793 - val_loss: 0.0440
Epoch 4/20
2532/2532 ━━━━━━━━━━━━━━━━━━━━ 83s 18ms/step - loss: 0.0788 - val_loss: 0.0440
Epoch 5/20
2532/2532 ━━━━━━━━━━━━━━━━━━━━ 80s 17ms/step - loss: 0.0791 - val_loss: 0.0439
Epoch 6/20
2532/2532 ━━━━━━━━━━━━━━━━━━━━ 82s 17ms/step - loss: 0.0788 - val_loss: 0.0440
Epoch 7/20
2532/2532 ━━━━━━━━━━━━━━━━━━━━ 43s 17ms/step - loss: 0.0789 - val_loss: 0.0439
Epoch 8/20
2532/2532 ━━━━━━━━━━━━━━━━━━━━ 82s 17ms/step - loss: 0.0791 - val_loss: 0.0440
Epoch 9/20
2532/2532 ━━━━━━━━━━━━━━━━━━━━ 45s 18ms/step - loss: 0.0791 - val_loss: 0.0440
Epoch 10/20
2532/2532 ━━━━━━━━━━━━━━━━━━━━ 45s 18ms/step - loss: 0.0790 - val_loss: 0.0442
Epoch 11/20
2532/2532 ━━━━━━━━━━━━━━━━━━━━ 44s 18ms/step - loss: 0.0789 - val_loss: 0.0443
Epoch 12/20
2532/25